In [ ]:
from utils import extract_image_info, split_channels, segment_nuclei, extract_intensities, classify_cells, convert_labels_to_edges, plot_segmentation 
from pathlib import Path
from cellpose import models

In [ ]:
# -------- DEFINE YOUR ANALYSIS PARAMETERS BELOW --------- #

# This is the default Cellpose model to segment nuclei, it leverages a GPU if its available
model = models.Cellpose(gpu=True, model_type="nuclei")

# Diameter in pixels of the nuclei present in your image, helps Cellpose to adjust nuclei mask predictions
cellpose_nuclei_diameter = 40

# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 5

# -------- CHOOSE YOUR CELL CLASSIFICATION METHOD --------- #

# You can choose to classify your cells as positive or negative for a certain marker based on a thresholding method (values above a certain mean intensity) 
# or on a percentage basis (cells in the top 10% of mean intensity for said marker).

# Type below your classifier choice ("threshold" or "percentage"). You can do it for both markers or 

classifier = "threshold"

# If you choose threshold:

# Any nuclei containing a higher than threshold value for signal intensity will be considered as a + cell for said marker
h2a_threshold = 40 # Mean intensity within the nucleus mask (h2a signal)
cfos_threshold = 20 # Mean intensity within the nucleus mask (cfos signal)

# If you choose percentage:

# Any nuclei containing a mean intensity signal within the top (top_percentage)% and (ratio)times above the mean intensity of the marker within all nuclei
# will be considered as a + cell for said marker. The ratio is implemented to avoid situation when all cells in an image have a similar average signal intensity
# but none are positive (i.e. background), given the nature of these samples we know there is no situation where most of the cells are positive.

h2a_top_percentage = 10
cfos_top_percentage = 10
h2a_ratio = 1.5
cfos_ratio = 1.5

# -------- DEFINE YOUR RAW_DATA DIRECTORY --------- #

# Scan for all images present in the data folder and add their Paths to the images list
data_path = Path("./data")
images = []

for file_path in data_path.glob("**/**/**/*.lsm"):
    images.append(file_path)


# ------- ANALYSIS PIPELINE --------- #

# Run analysis pipeline over all images and show segmentation results    
for image in images:

    # Extract filename, region, mouse and IHC round
    filename, region, mouse_id, ihc_round = extract_image_info(image)

    # Split channels
    nuclei_img, h2a_img, cfos_img = split_channels(image)

    # Segment nuclei
    nuclei_masks = segment_nuclei(nuclei_img, gaussian_sigma, model, cellpose_nuclei_diameter)

    # Morphological and intensity measurements
    merged_df = extract_intensities(nuclei_masks, h2a_img, cfos_img, filename, region, mouse_id, ihc_round)
    
    # Select H2A and CFOS positive cells based on mean_intensity thresholds, return a mask of + cells
    h2a_nuclei_labels, cfos_nuclei_labels, double_pos_nuclei_labels, h2a_pos_labels, cfos_pos_labels, double_pos_labels = classify_cells(merged_df, nuclei_masks, h2a_threshold, cfos_threshold)
    
    # Visualize nuclei labels as just the perimeter for ease of interpretation    
    nuclei_edges = convert_labels_to_edges(nuclei_masks, radius=1)
    
    print(f"Filepath: {image}")
    print(f"H2A_+ labels: {h2a_pos_labels}")
    print(f"CFOS_+ labels: {cfos_pos_labels}")
    print(f"H2A & CFOS_+ labels: {double_pos_labels}")
    
    # Visualize the segmentation results on a per image basis
    plot_segmentation(nuclei_img, nuclei_edges, h2a_img, h2a_nuclei_labels, cfos_img, cfos_nuclei_labels, double_pos_nuclei_labels)